### <u> Generate the Planetary Orbit sonifications used in the "_Audible Universe_" planetarium show </u>

**First, import relevant modules:**

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import SingleObject
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Synthesizer
import IPython.display as ipd
import os
from scipy.interpolate import interp1d

**Collate the notes we are using to represent each planet and their orbital periods, as well as the length of each sonification**

Then, combine these into dictionaries so they can be easily indexed

In [ ]:
datafile = "../data/datasets/landfrac.txt"
data = np.genfromtxt(datafile)

longitude = data[:,0]
waterfrac = 1-data[:,1]

startlong = 180-(96 + 15./60 + 2.2/3600)
print(startlong-180)
# we travel backwards in longitude per the earth's rotation
longgrid = (np.linspace(startlong,720+startlong,2599)%360 - 180.)[::-1] 
wfrac = interp1d(longitude, waterfrac)

wfracgrid = wfrac(longgrid)*0.75 + 0.15
timegrid = np.linspace(0,1,wfracgrid.size)

plt.plot(timegrid, wfracgrid)
plt.show()


In [ ]:
# chord representing the earth (a Gbsus7 chord)
notes = [['B4']] #[['Gb3', 'Db4', 'E4', 'B5']]

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

length = 60.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.modify_preset({'filter':'on'}) 
print(generator.preset)

**Render sonification for specified planet...**

In [ ]:
score =  Score(notes, length)
print(2001*1322)
# volume swell is directly ahead
data = {'cutoff':wfracgrid,
        'time_evo':timegrid,
        'pitch':np.ones(timegrid.size)}

# set up source
events = SingleObject(data.keys())
events.fromdict(data)
events.apply_mapping_functions()

soni = Sonification(score, events, generator, system)
soni.render()

**Listen to and plot the waveforms from the sonification:**

In [ ]:
# I should adapt this to an inbuilt function of the sonification class at some point...
time = soni.out_channels['0'].samples / soni.out_channels['0'].samprate
for i in range(len(soni.out_channels)):
    plt.plot(time[::20], soni.out_channels[str(i)].values[::20]-1.2*i, label=soni.channels.labels[i])

plt.xlabel('Time (s)')
plt.ylabel('Relative Amplitude')
plt.legend(frameon=False, loc=5)
plt.xlim(-time[-1]*0.05,time[-1]*1.2)
for s in plt.gca().spines.values():
    s.set_visible(False)
plt.gca().get_yaxis().set_visible(False)

if len(soni.channels.labels) == 1:
    # we have used 441000 Hz everywhere above as standard, but to quickly hear the sonification sped up / slowed down,
    # you can modify the 'rate' argument below (e.g. multiply by 0.5 for half speed, by 2 for double speed, etc)
    outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['0'].values]).T
else:
    outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['1'].values]).T
ipd.Audio(outfmt,rate=soni.out_channels['0'].samprate, autoplay=False)

**Combine and save sonification to a multi-channel wav** 

NOTE: Change `"../../FILENAME.wav"` to your filepath of choice

In [ ]:
soni.save_combined("../../FILENAME.wav", True)

In [ ]:
data = {'phi':[],
        'theta':[], # constant theta of 90 deg
        'pitch':[],     # constant pitch
        'volume':[],
        'time_evo': []}

n = 10000

for k in planets:
    panphase = np.random.random()*360.
    # volume swell is directly ahead
    volphase = panphase + 90

    # setup score
    score =  Score(chorddict[planet], lendict[planet])

    # data dict
    orbits_per_sonification = lendict[planet]/perioddict[planet]
    orbital_phi = (np.linspace(0,orbits_per_sonification,n)%1)*360

    data['phi'].append(orbital_phi + panphase)
    data['theta'].append(np.ones(n)*90.) 
    data['pitch'].append(np.ones(n))
    data['volume'].append(np.sin((orbital_phi + volphase) * np.pi/180.)*0.4+0.6)
    data['time_evo'].append(np.linspace(0,1,n))

# set up source
events = SingleObject(mapvals.keys())
events.fromdict(data)
events.apply_mapping_functions(mapvals, maplims)

soni = Sonification(score, events, sampler, system)
soni.render()